#### Display image

In [ ]:
# not required
def display(image:np.array, title="image"):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
from math import inf
import cv2
import numpy as np
import pandas as pd

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import threading as thd

mp_drawing  = mp.solutions.drawing_utils

## Face bounds detector

In [ ]:
# https://mediapipe.readthedocs.io/en/latest/solutions/face_detection.html

class FaceDetector:
    '''
    FaceDetector is used to get the 'bounds' for a face.
    'bounds' are used to crop the image befor eseonding it for face/iris landmark detection.
    '''
    def __init__(self):
        # creating detector object
        with open(r".\data\blaze_face_short_range.tflite", "rb") as model_file:
            model_data = model_file.read()
        
        options = vision.FaceDetectorOptions(
            base_options = python.BaseOptions(model_asset_buffer=model_data),
            running_mode = vision.RunningMode.IMAGE
        )
        
        self.face_detector = vision.FaceDetector.create_from_options(options)
        
    def detect_face_bounds(self, image:np.array) -> tuple:
        # convert nump image to mediapipe format
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        
        # detect face in image
        self.face_detection_result = self.face_detector.detect(mp_image)
    
        # if face detected, draw on image and return bounds
        if self.face_detection_result.detections:
            bbox = self.face_detection_result.detections[0].bounding_box
            self.draw_bounds(image, bbox)
            return self.expand_bounds(bbox, scale=2)      # double width and height

        print("No face detected")
        return None

    
    def draw_bounds(self, image:np.array, bbox) -> tuple:
        height, width, _ = image.shape
        start_point = bbox.origin_x, bbox.origin_y
        end_point   = bbox.origin_x + bbox.width, bbox.origin_y + bbox.height
        
        cv2.rectangle(image, start_point, end_point, (255,0,0), 3)

    
    def expand_bounds(self, bbox, scale:int=2):
        x = max(0, round( bbox.origin_x - bbox.width*(scale-1)/2 )  )
        y = max(0, round( bbox.origin_y - bbox.height*(scale-1)/2 ) )

        #print(x,y)
        bbox.x = x
        bbox.y = y
        bbox.width  = round(scale*bbox.width)
        bbox.height = round(scale*bbox.height)

        return bbox

## Facemesh detector

In [ ]:
class FaceMeshDetector:
    def __init__(self):
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh    = self.mp_face_mesh.FaceMesh()

    def detect(self, image):
        results = self.face_mesh.process(image)
        self.draw_face_landmarks(image, results)   # OPTIONAL
        
        return results
    
    def draw_face_landmarks(self, image, results):
        if not results.multi_face_landmarks:
            return
        for face_landmark in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image = image,
                landmark_list = face_landmark,
                connections   = self.mp_face_mesh.FACEMESH_TESSELATION,
    
                landmark_drawing_spec = mp_drawing.DrawingSpec(
                    color=(255,0,0),
                    thickness=0,
                    circle_radius=1),
                
                connection_drawing_spec = mp_drawing.DrawingSpec(
                    color=(255,255,255),
                    thickness=1,
                    circle_radius=1)    )

## Data Accumulater

In [ ]:
'''
- eyebrow (L&R)                x2
- lips    (L&R)                x2
- face: left right top bottom  x4
- body                         x8   < change here
- iris    (L&R)                x2
= 18 columns : x and y         = 36 features
'''

columns0 = ['lip_L', 'lip_R', 'brow_L', 'brow_R', 'iris_L', 'iris_R'] + [f"face{i}" for i in range(4)]  + [f"body{i}" for i in range(8)]
columns  = list()
for col in columns0:
    columns.append(col+'x')
    columns.append(col+'y')
columns.append('TRUTH')
TRAINING_FEATURES = pd.DataFrame(columns=columns)
TESTING_FEATURES  = pd.DataFrame(columns=columns)

# https://raw.githubusercontent.com/google/mediapipe/master/mediapipe/modules/face_geometry/data/canonical_face_model_uv_visualization.png
LANDMARKS_LOC = {
    'brow_L'     : {107, 66, 105, 63, 70, 46, 53, 52, 65, 55},
    'brow_R'     : {336, 285, 296, 295, 334, 282, 293, 283, 276, 300},
    'lip_L'      : {78, 191, 80, 81, 82, 95, 88, 178, 87},
    'lip_R'      : {308, 415, 324, 310, 318, 311, 402, 312, 317},
    'face0'      : {54, 68, 103, 104, 108, 69, 67, 10, 151, 338, 337, 397, 333, 332, 298, 284, 251, 301, 21, 71, 109, 297, 299},                                                                       #forehead
    'face1'      : {18, 32, 83, 140, 148, 152, 171, 175, 176, 199, 200, 201, 208, 262, 313, 369, 377, 396, 400, 421, 428},                                                       #chin
    'face2'      : {36, 50, 58, 93, 101, 111, 116, 117, 118, 123, 132, 137, 138, 147, 172, 177, 186, 187, 192, 203, 205, 206, 207, 212, 213, 214, 215, 216, 227, 228, 234},      #left_face
    'face3'      : {266, 280, 288, 323, 330, 340, 345, 346, 347, 352, 361, 366, 367, 376, 397, 401, 410, 411, 416, 423, 425, 426, 427, 432, 433, 434, 435, 436, 447, 448, 454}   #right_face
}
    
# video intervals where the subject answers
INTERVALS = (
    (90,  140, 1),
    (170, 220, 1),
    (250, 340, 0),
    (370, 450, 1),    # TESTING INTERVAL
    (480, 540, 1),
    (600, 660, 0),
    (inf, inf,0) )    # prevents detection on remainder video

TEST_INTERVAL = 4-1

TRAINING_FEATURES.shape

In [ ]:

class CaptureData:
    def __init__(self):
        

## 2) Live video detecting

In [ ]:
face_bounds_detect = FaceDetector()
face_mesh_detector = FaceMeshDetector()

In [ ]:
# cam = cv2.VideoCapture(0)
cam = cv2.VideoCapture(r".\previous-MLcode\Train_video.mp4")

# get face bounds in image
bbox = None
while bbox is None:
    _, frame = cam.read()
    bbox = face_bounds_detect.detect_face_bounds(frame)  # keep this before while True


# process video
while True:
    _, frame = cam.read()
    if not _: break

    cropx256 = cv2.resize(frame[bbox.y : bbox.y+bbox.width, bbox.x : bbox.x+bbox.height], (512,512))
    
    results = face_mesh_detector.detect(cropx256)

    cv2.imshow("cam", cropx256)
        
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()